### Notebook para buscar utterances similares. Se importa un archivo que contiene utterances de los cuales se quiere buscar parecidos y otro en el que se buscan.


In [ ]:
archivo_csv_importar = 'IR_portabilidad.csv' #este archivo debería contener utterances de los cuales se quiere buscar similares
nombreColumnaTexto = "utterances" #indicar el nombre de la columna que contiene los utterances

archivo_csv_importar_ = ' ' #archivo que contiene los utterances sobre los que se quiere buscar
nombreColumnaTexto_ = " " #indicar el nombre de la columna que contiene los utterances



In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
# import random 
from random import sample
import os, ibm_db, ibm_db_dbi as dbi
import operator
from collections import Counter
import string
import re

pd.set_option("display.max_rows", 600)
pd.set_option("display.max_columns", 500)
pd.set_option("max_colwidth", 400)

In [ ]:
pip install transformers==4.15.0

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install -U sentence-transformers #instalar librería para vectorizar textos con modelos preentrenados

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install nltk #instalar nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sentence_transformers import SentenceTransformer, util #se importan
import nltk #se importa

In [ ]:
#### MODELO PARA VECTORIZAR LOS TEXTOS
#https://github.com/UKPLab/sentence-transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

In [ ]:
import io



def generarExcel(listaDfs,listaPestaniasExcel,nombreArchivo):

    #se genera un excel y se exporta al ICOS.
    #listaDfs tiene que coincidir (en tamaño) con listaPestaniasExcel ya que cada pestaña va a contener cada DF.
    #nombreArchivo es el del Excel
    
    output = io.BytesIO()
    writer = pd.ExcelWriter(output, engine='xlsxwriter')
    
    for dff, pest in zip(listaDfs, listaPestaniasExcel):
        dff.to_excel(writer,sheet_name=pest,index=False)
        
    for i,x in zip(writer.sheets.keys(),listaDfs ):
        # Get the xlsxwriter workbook and worksheet objects.
        workbook = writer.book
        worksheet = writer.sheets[i]

        # Get the dimensions of the dataframe.
        (max_row, max_col) = x.shape

        # Create a list of column headers, to use in add_table().
        column_settings = [{'header': column} for column in x.columns]

        # Add the Excel table structure. Pandas will add the data.
        worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

        # Make the columns wider for clarity.
        worksheet.set_column(0, max_col - 1, 12)
        
        # Wrap text for the headers
        header_format = workbook.add_format({'text_wrap': True, 'align': 'center', 'valign': 'vcenter'})
        worksheet.set_row(0, None, header_format)

        # Create a format object to use in the conditional formatting.
        numeric_cell_format = workbook.add_format({'align': 'center'})
        text_cell_format = workbook.add_format({'align': 'left'})

        # Apply conditional formatting to each cell in the table.
        for row in range(max_row):
            for col in range(max_col):
                # If the length of the cell text is greater than 16, apply the text wrap format.
                # Otherwise, apply the text center format.
                if len(str(x.iat[row, col])) > 16:
                    worksheet.write(row + 1, col, x.iat[row, col], text_cell_format)
                else:
                    worksheet.write(row + 1, col, x.iat[row, col], numeric_cell_format)
        
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

    xlsx_data = output.getvalue()

    #falta linea para exportar
    
    
# limpiar puntuaciones y stopwords
punctuation = string.punctuation + "«»“”‘’…—" #se definen signos de puntuación para poder borrar de los textos

def normalize(s): 
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements: #se sacan los tildes
        s = s.replace(a, b).replace(a.upper(), b.upper())
        
    s = s.translate(str.maketrans('', '', punctuation)).lower()#.split() #se sacan los signos de puntuación
    
    return s

def buscarUtterancesSimilares(df_1, df_2,nombreColumnaTexto,nombreColumnaIntencion):
    print(f"Empieza {datetime.now()}")
    todo = []
    b = df_1.FCD_COG_INPUT_TEXT.to_list()
    X = model.encode(b)
    for index in df_2.index:    
        sentence__ = model.encode(df_2[nombreColumnaTexto].iloc[index]) #se vectoriza el utterance

        cos_sim = util.cos_sim(sentence__, X) #se compara un determinado utterance con todos los del dataset. Cuando el resultado es cercano a 1 (comparando dos utterances) eso significa que son similares

        winners = []
        for arr in cos_sim: #se recorre cada resultado y se guarda en una lista, también la utterance
            for i, each_val in enumerate(arr):
                if each_val >= 0.70:
                    winners.append([b[i],each_val.item(),df_1.index[i],df_1.FCD_COG_CONTEXT_CALL_ID.iloc[i],df_1.FCD_COG_CONTEXT_CALLREASON.iloc[i],df_2[nombreColumnaTexto].iloc[index],df_2[nombreColumnaIntencion].iloc[index]]) 

        todo = todo + winners


    utterancesEntrar=pd.DataFrame(todo) #se transforma la lista a df
    utterancesEntrar.rename(columns={1:"similitud",0:"utterance",3:"callId",4:"Intecion Inicial",5:"similar a",6:"intencionDelUtteranceOriginal"},inplace=True) #nombre de columnas
    mayor_similitud=pd.DataFrame(utterancesEntrar.groupby(["callId","utterance"])["similitud"].max()).reset_index().copy() #cuando un utterance se repite, se selecciona el de mayor similitud
    utterancesEntrar=pd.merge(mayor_similitud, utterancesEntrar, left_on=["callId","utterance","similitud"], right_on=["callId","utterance","similitud"],how="left") 
    utteranceNoEntrar=df_1[~ df_1.index.isin(utterancesEntrar[2])].copy() #los utterances que no tienen similitud con ninguno de entrenamiento
    
    print(f"Termina {datetime.now()}")
    
    return utterancesEntrar, utteranceNoEntrar

In [ ]:
#se trae el archivo sobre el cual se quieren buscar utterances
utterancesProdd = project_actual.get_file(archivo_csv_importar_) 
utterancesProdd = pd.read_csv(utterancesProdd)

utterancesProdd[nombreColumnaTexto_] = utterancesProdd[nombreColumnaTexto_].apply(lambda x: normalize(x)) #se normalizan

In [ ]:
#se trae el archivo que contiene los utterances de los cuales se quiere buscar similares
utterances = project_actual.get_file(archivo_csv_importar) 
utterances = pd.read_csv(utterances)

utterances[nombreColumnaTexto] = utterances[nombreColumnaTexto].apply(lambda x: normalize(x)) #se normalizan

In [ ]:
utterancesSimilares, utteranceNoSimilares = buscarUtterancesSimilares(utterancesProdd,utterances,nombreColumnaTexto,nombreColumnaIntencion)
#se realiza la búsqueda para encontrar los similares

Empieza 2023-02-07 18:25:00.437538
Termina 2023-02-07 18:28:14.155548


In [ ]:
utterancesSimilares.drop(columns=[2],inplace=True) #se dropea una columna que está de más

utterancesSimilares.drop_duplicates(inplace=True) #se dropean los duplicados

utterancesSimilares.reset_index(inplace=True) #se resetea el índice

utterancesSimilares['similitud'] = utterancesSimilares['similitud'].round(3) #se redondea el valor de similitud a 3 decimales

utterancesSimilares #se printea

,index,callId,utterance,similitud,Intecion Inicial,similar a,intencionDelUtteranceOriginal
0,0,6RC968IHND58FFBIU6QUILASR001936L,informacion de portabilidad,0.771,-,consultar como es la portabilidad en la compañia,portabilidad
1,1,6RC968IHND58FFBIU6QUILASR001936L,portabilidad numerica,0.761,-,como puedo hablar con portabilidad numerica,portabilidad
2,2,6RC968IHND58FFBIU6QUILASR00193SF,quiero hacer el cambio de planes del del telefono,0.723,-,quiero cambiar de compañia de telefono,portabilidad
3,3,6RC968IHND58FFBIU6QUILASR001953K,necesito pasar eh un numero de telefono a un personal,0.707,-,queria saber si se puede pasar un numero de personal a un numero de claro,portabilidad
4,4,6RC968IHND58FFBIU6QUILASR001953K,si quiero volver a tener linea eh llamaba para pedir la linea de nuevo no es de este telefono de otro numero,0.704,-,llamo porque quiero cambiar mi linea,portabilidad
...,...,...,...,...,...,...,...
1048,1079,I4LAOTSU0P4CHA2QJ3L16PP3VK02272J,queria este saber si se puede hacer la localizacion de un numero de telefono de personal,0.709,-,queria saber si se puede pasar un numero de personal a un numero de claro,portabilidad
1049,1080,I4LAOTSU0P4CHA2QJ3L16PP3VK02287V,cambiar de linea a personal,0.741,-,queria cambiar un numero de claro a personal,portabilidad
1050,1081,I4LAOTSU0P4CHA2QJ3L16PP3VK02287V,queria saber para pasar otra linea a personal,0.774,-,queria saber si se puede pasar un numero de personal a un numero de claro,portabilidad
1051,1082,I4LAOTSU0P4CHA2QJ3L16PP3VK0228G3,si quiero una promocion con gigas con mas digas si es factible hoy a cambiar de empresa,0.705,IR_portabilidad,me paso a otra compañia porque me ofrecen planes mucho mas economicos,portabilidad


In [ ]:
# generarExcel([utterancesSimilares,utterancesSimilares],["a","b"],"prueba") #por si se quiere exportar con formato de tabla dinámica 